In [1]:
import sys
sys.path.append('/host/d/Github')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import SimpleITK as sitk
import Aorta_segmentation_ZC.functions_collection as ff
import Aorta_segmentation_ZC.Data_processing as Data_processing

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
main_path = '/host/d/Data/CTA/original_data'

### process AVT

In [4]:
case_list = ff.find_all_target_files(['*/*'], os.path.join(main_path, 'AVT'))
print('Number of cases found:', len(case_list))

Number of cases found: 56


In [ ]:
for i in range(0, len(case_list)):
   patient_id = os.path.basename(case_list[i])
   patint_subclass = os.path.basename(os.path.dirname(case_list[i]))
   patient_class = 'AVT'


   if os.path.exists(os.path.join(case_list[i], 'img.nii.gz')):
      print('NIfTI files already exist, skip case.')
      continue
   
   # assert there are two files in the folder and one has "seg"
   files = ff.find_all_target_files(['*.nrrd'], case_list[i])
   assert len(files) == 2, 'There should be two files in the folder: '
   have_seg = False
   for j in range(0, len(files)):
      if 'seg' in os.path.basename(files[j]):
        have_seg = True
        data, affine,_,_,_ = ff.load_nrrd(files[j])
        data = np.rollaxis(data,0,3)
        data = np.flip(np.flip(np.rollaxis(data,0,2),axis = 1),axis = 0)
        nii_path = os.path.join(case_list[i], 'seg.nii.gz')
        nii_image = nb.Nifti1Image(data, affine)
        nb.save(nii_image, nii_path)
      else:
        data, affine,_,_,_ = ff.load_nrrd(files[j])
        data = np.rollaxis(data,0,3)
        data = np.flip(np.flip(np.rollaxis(data,0,2),axis = 1),axis = 0)
        nii_path = os.path.join(case_list[i], 'img.nii.gz')
        nii_image = nb.Nifti1Image(data, affine)
        nb.save(nii_image, nii_path)
   assert have_seg, 'One file should be segmentation file with "seg" in the filename'

    
    
             


### process AD

In [3]:
case_list = ff.find_all_target_files(['AD/*'], os.path.join(main_path, 'AD'))
print('Number of cases found:', len(case_list))

Number of cases found: 40


In [25]:
for i in range(0, len(case_list)):
   patient_id = os.path.basename(case_list[i])
   patint_subclass = os.path.basename(os.path.dirname(case_list[i]))
   patient_class = 'AD'
   print('patient class:', patient_class, ' subclass:', patint_subclass, ' id:', patient_id)

   if os.path.exists(os.path.join(case_list[i], 'seg.nii.gz')):
      print('NIfTI files already exist, skip case.')
      continue

   files = ff.find_all_target_files(['*.nrrd'], case_list[i])
   assert len(files) == 3, 'There should be three files in the folder: '
   have_seg = 0
   for j in range(0, len(files)):
      if 'seg' in os.path.basename(files[j]):
        data, affine,_,_,_ = ff.load_nrrd(files[j])
        data = np.rollaxis(data,0,3)
        data = np.flip(np.flip(np.rollaxis(data,0,2),axis = 1),axis = 0)
        assert np.unique(data).size == 2, 'Segmentation should be binary'
        if have_seg == 0:
            seg = np.zeros_like(data)
        if 'true' in os.path.basename(files[j]):
            seg[data>0] = 1
        else:
            seg[data>0] = 2
        have_seg += 1
      else:
        data, affine,_,_,_ = ff.load_nrrd(files[j])
        data = np.rollaxis(data,0,3)
        data = np.flip(np.flip(np.rollaxis(data,0,2),axis = 1),axis = 0)
        nii_path = os.path.join(case_list[i], 'img.nii.gz')
        nii_image = nb.Nifti1Image(data, affine)
        nb.save(nii_image, nii_path)

   assert have_seg == 2, 'There should be two segmentation files with "seg" in the filename'
   # save combined seg
   nii_path = os.path.join(case_list[i], 'seg.nii.gz')
   nii_image = nb.Nifti1Image(seg, affine)
   nb.save(nii_image, nii_path)



    
             


patient class: AD  subclass: AD  id: cta01s
NIfTI files already exist, skip case.
patient class: AD  subclass: AD  id: cta02s
patient class: AD  subclass: AD  id: cta03s
patient class: AD  subclass: AD  id: cta04s
patient class: AD  subclass: AD  id: cta05s
patient class: AD  subclass: AD  id: cta06s
patient class: AD  subclass: AD  id: cta07s
patient class: AD  subclass: AD  id: cta08s
patient class: AD  subclass: AD  id: cta09s
patient class: AD  subclass: AD  id: cta10s
patient class: AD  subclass: AD  id: cta11s
patient class: AD  subclass: AD  id: cta12s
patient class: AD  subclass: AD  id: cta13s
patient class: AD  subclass: AD  id: cta14s
patient class: AD  subclass: AD  id: cta15s
patient class: AD  subclass: AD  id: cta16s
patient class: AD  subclass: AD  id: cta17s
patient class: AD  subclass: AD  id: cta18s
patient class: AD  subclass: AD  id: cta19s
patient class: AD  subclass: AD  id: cta20s
patient class: AD  subclass: AD  id: cta21s
patient class: AD  subclass: AD  id: c

### print out the data info

In [4]:
case_list = ff.find_all_target_files(['*/*/*'], main_path)
print('Number of cases found:', len(case_list))

Number of cases found: 96


In [5]:
results = []
for i in range(0, len(case_list)):
    patient_id = os.path.basename(case_list[i])
    patint_subclass = os.path.basename(os.path.dirname(case_list[i]))
    patient_class = os.path.basename(os.path.dirname(os.path.dirname(case_list[i])))
    # print('patient class:', patient_class, ' subclass:', patint_subclass, ' id:', patient_id)
    image_file = os.path.join(case_list[i], 'img.nii.gz')

    img_nii = nb.load(image_file)
    spacing = img_nii.header.get_zooms()
    spacing_x, spacing_y, spacing_z = spacing
    shape = img_nii.shape
    shape_x, shape_y, shape_z = shape

    results.append([patient_class, patint_subclass, patient_id, shape_x, shape_y, shape_z, spacing_x, spacing_y, spacing_z])

    df = pd.DataFrame(results, columns=['class', 'subclass', 'id', 'shape_x', 'shape_y', 'shape_z', 'spacing_x', 'spacing_y', 'spacing_z'])
    df.to_excel(os.path.join(os.path.dirname(main_path), 'Patient_lists/original_data_info.xlsx'))


### make z-slice thickness at most 0.8mm

In [ ]:
patient_list = pd.read_excel(os.path.join(os.path.dirname(main_path), 'Patient_lists/original_data_info.xlsx'))
for i in range(0, patient_list.shape[0]):
    row = patient_list.iloc[i]
    spacing_z = row['spacing_z']
    spacing_x = row['spacing_x']

    # if os.path.exists(os.path.join(main_path, row['class'], row['subclass'], row['id'], 'seg_resampled.nii.gz')):
    #     print('Resampled files already exist, skip case.')
    #     continue

    if spacing_z>0.8:
        print('Patient ID:', row['id'], ' spacing_z:', spacing_z)
    
        img_file = os.path.join(main_path, row['class'], row['subclass'], row['id'], 'img.nii.gz')
        seg_file = os.path.join(main_path, row['class'], row['subclass'], row['id'], 'seg.nii.gz')
        
        # resample to 0.8mm slice thickness
        # do image first
        img_nifti = nb.load(img_file)
        img = img_nifti.get_fdata()
        img_resampled_nifti = Data_processing.resample_nifti(img_nifti, order = 3, mode = 'nearest', cval = np.min(img), in_plane_resolution_mm = spacing_x, slice_thickness_mm = 0.8, number_of_slices = None)

        header = img_resampled_nifti.header
        affine = img_resampled_nifti.affine
        img_resampled_data = img_resampled_nifti.get_fdata()
        img_resampled_data = np.round(img_resampled_data).astype(np.int16)
        img_resampled_nifti = nb.Nifti1Image(img_resampled_data, affine, header)

        new_filename = os.path.join(main_path, row['class'], row['subclass'], row['id'], 'img_resampled.nii.gz')
        nb.save(img_resampled_nifti, new_filename)

        # do segmentation
        seg_nifti = nb.load(seg_file)
        seg_resampled_nifti = Data_processing.resample_nifti(seg_nifti, order = 0, mode = 'nearest', cval = 0, in_plane_resolution_mm = spacing_x, slice_thickness_mm = 0.8, number_of_slices = None)
        header = seg_resampled_nifti.header
        affine = seg_resampled_nifti.affine
        seg_resampled_data = seg_resampled_nifti.get_fdata()
        seg_resampled_data = np.round(seg_resampled_data).astype(np.int16)
        seg_resampled_nifti = nb.Nifti1Image(seg_resampled_data, affine, header)

        new_seg_filename = os.path.join(main_path, row['class'], row['subclass'], row['id'], 'seg_resampled.nii.gz')
        nb.save(seg_resampled_nifti, new_seg_filename)
    else:
        # copy
        import shutil
        img_file = os.path.join(main_path, row['class'], row['subclass'], row['id'], 'img.nii.gz')
        seg_file = os.path.join(main_path, row['class'], row['subclass'], row['id'], 'seg.nii.gz')
        new_img_file = os.path.join(main_path, row['class'], row['subclass'], row['id'], 'img_resampled.nii.gz')
        new_seg_file = os.path.join(main_path, row['class'], row['subclass'], row['id'], 'seg_resampled.nii.gz')    
        shutil.copyfile(img_file, new_img_file)
        shutil.copyfile(seg_file, new_seg_file)

    
  

Patient ID: cta01s  spacing_z: 1.5
Patient ID: cta02s  spacing_z: 2.0
Patient ID: cta03s  spacing_z: 1.5
Patient ID: cta04s  spacing_z: 2.0
